In [2]:
# sample code for writing BD5 data (line entity)

import sys
import tables
from tables import *
import numpy as np
import cv2
import warnings

# for supression of natural name warning
def write_unsafe_attrs(node, attrs):
    original_warnings = list(warnings.filters)
    warnings.simplefilter('ignore', tables.NaturalNameWarning)
    for name, value in attrs:
        node._f_setAttr(name, value)
        warnings.filters = original_warnings

# class definition
class ScaleUnit(IsDescription):
    dimension = StringCol(8, dflt="", pos=0)
    x = Float64Col(dflt=0, pos=1)
    y = Float64Col(dflt=0, pos=2)
    sUnit = StringCol(16, dflt="", pos=3)
    t = Float64Col(dflt=0, pos=4)
    tUnit = StringCol(16, dflt="", pos=5)

class ObjectDef(IsDescription):
    oID = UInt16Col(dflt=0, pos=0)
    name = StringCol(128, dflt="", pos=1)

class Data(IsDescription):
    ID = StringCol(32, dflt="", pos=0)
    t = Int32Col(dflt=0, pos=1)
    entity = StringCol(8, dflt="", pos=2)
    x = UInt16Col(dflt=0, pos=3)
    y = UInt16Col(dflt=0, pos=4)

# input and output
img_path = "./bw_sample.tif"
hdf5_out = "./sample_line_bd5.h5"

# preprocessing
img = cv2.imread(img_path)
imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(imgray,127,255,0)

# find contours of ROIs
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

contours = [np.squeeze(cnt, axis=1) for cnt in contours]

# output to bd5 file
h5file = open_file(hdf5_out, mode="w", title="bd5 file")
group = h5file.create_group('/', 'data', 'data group')

# for scaleUnit
scaleunit = h5file.create_table('/data', 'scaleUnit', ScaleUnit, "definition of scale and unit")
su = scaleunit.row
su['dimension'] = "2D"
su['x'] = 0.1
su['y'] = 0.1
su['sUnit'] = "micrometer"
su['t'] = 1
su['tUnit'] = "minute"
su.append()
scaleunit.flush()

# for objectDef
objectdef = h5file.create_table('/data', 'objectDef', ObjectDef, "object definition")
od = objectdef.row
od['oID'] = 0
od['name'] = "nucleus"
od.append()
objectdef.flush()

# series index
series = h5file.create_group('/data', '0', 'time index 0')

# store xy coordinates to dataset table
table = h5file.create_table('/data/0', '0', Data, "dataset")
xyz = table.row
for i, cnt in enumerate(contours):
    for j in cnt:
        xyz['ID']= "o" + '%06d' % (i)
        xyz['t'] = 0
        xyz['entity'] = 'line'
        xyz['x'] = j[0]
        xyz['y'] = j[1]
        xyz.append()
    # for closed line
    xyz['ID'] = "o" + '%06d' % (i)
    xyz['t'] = 0
    xyz['entity'] = 'line'
    xyz['x'] = cnt[0, 0]
    xyz['y'] = cnt[0, 1]
    xyz.append()
table.flush()

# how to read a dataset (test code)
# table2 = h5file.get_node('data', '0').read()

h5file.close()